In [1]:
from init import *

In [2]:
def BuildModel_Simple():
    '''Geometrical brownian motion with hidden drift and diffusion parameters.
    The measurements are zero coupon bonds.
    '''
    m = Model()
    m.TimeStart = 0
    m.TimeSteps = 1
    m.NumPaths = 1
    m.Add(IndependentGaussian())
    bm_a = m.Add(BrownianMotion(Parameter(value=0,step=0.1,min=-100,max=100),0,2)) # start, drift, diffusion
    m.Add(CorrelatedGaussian(Parameter(value=-1,min=-1,max=1,step=0.0),bm_a.GetStateNumber(),bm_a.GetStateNumber()+1))
    bm_b = m.Add(BrownianMotion(0,0,Parameter(value=2,step=0.2))) # start, drift, diffusion
    assert (bm_a.GetStateNumber()+1) == bm_b.GetStateNumber()
    return m

In [3]:
import time
model = BuildModel_Simple()
model.NumPaths = 10000
model.TimeSteps = 100
model.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    model.evaluations.append(EvaluationPoint(0,t))
model.json()

'{"TimeStart": 0, "TimeSteps": 100, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian", "refs": []}, {"name": "BrownianMotion", "start": {"value": 0, "step": 0.1, "min": -100, "max": 100}, "args": [0, 2]}, {"name": "CorrelatedGaussian", "args": [{"value": -1, "step": 0.0, "min": -1, "max": 1}], "refs": [0, 1]}, {"name": "BrownianMotion", "start": 0, "args": [0, {"value": 2, "step": 0.2}]}], "evaluations": [{"state": 0, "time": 1, "histograms": []}, {"state": 0, "time": 2, "histograms": []}, {"state": 0, "time": 5, "histograms": []}, {"state": 0, "time": 10, "histograms": []}], "RandomSeed": -1, "RunTimeoutSeconds": 16.7, "MemoryLimitKB": 64}'

In [4]:
result = requests.post(f'{server}/model',model.json())
er = EvaluationResults(result.json(),model)
er.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,10,10000,-0.029783,0.019869,1.986915,-0.006239
1,BrownianMotion,,1,1.0,10,10000,0.029783,0.019869,1.986915,0.006239
2,BrownianMotion,,0,2.0,20,10000,-0.042022,0.028308,2.830842,-0.021084
3,BrownianMotion,,1,2.0,20,10000,0.042022,0.028308,2.830842,0.021084
4,BrownianMotion,,0,5.0,50,10000,-0.083113,0.044524,4.452447,-0.002432
5,BrownianMotion,,1,5.0,50,10000,0.083113,0.044524,4.452447,0.002432
6,BrownianMotion,,0,10.0,100,10000,-0.046831,0.063024,6.302421,-0.022370
7,BrownianMotion,,1,10.0,100,10000,0.046831,0.063024,6.302421,0.022370


In [5]:
for u in model.updaters:
    print((u.GetEquationNumber(),u.GetStateNumber()),u.json())

(0, None) {"name": "IndependentGaussian", "refs": []}
(1, 0) {"name": "BrownianMotion", "start": {"value": 0, "step": 0.1, "min": -100, "max": 100}, "args": [0, 2]}
(2, None) {"name": "CorrelatedGaussian", "args": [{"value": -1, "step": 0.0, "min": -1, "max": 1}], "refs": [0, 1]}
(3, 1) {"name": "BrownianMotion", "start": 0, "args": [0, {"value": 2, "step": 0.2}]}


In [6]:
calib = ModelCalibration(model)
calib.Print()

2 calibration parameters:
BrownianMotion eq=1 narg=-1 {"value": 0, "step": 0.1, "min": -100, "max": 100}
BrownianMotion eq=3 narg=1 {"value": 2, "step": 0.2}


In [7]:
for cpar in calib.cpars:
    print(cpar)

eq=1 narg=-1 {"value": 0, "step": 0.1, "min": -100, "max": 100}
eq=3 narg=1 {"value": 2, "step": 0.2}


In [8]:
ModelCalibration(model).GetState()

[0, 2]

In [9]:
calib.SetCalibrationParameter(0,0.1)
print(calib.GetState())
calib.SetCalibrationParameter(0,0.11)
print(calib.GetState())
calib.SetCalibrationParameter(1,2.22)
print(calib.GetState())


[0.1, 2]
[0.11, 2]
[0.11, 2.22]


In [10]:
model.json()

'{"TimeStart": 0, "TimeSteps": 100, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian", "refs": []}, {"name": "BrownianMotion", "start": {"value": 0.11, "step": 0.1, "min": -100, "max": 100}, "args": [0, 2]}, {"name": "CorrelatedGaussian", "args": [{"value": -1, "step": 0.0, "min": -1, "max": 1}], "refs": [0, 1]}, {"name": "BrownianMotion", "start": 0, "args": [0, {"value": 2.22, "step": 0.2}]}], "evaluations": [{"state": 0, "time": 1, "histograms": []}, {"state": 0, "time": 2, "histograms": []}, {"state": 0, "time": 5, "histograms": []}, {"state": 0, "time": 10, "histograms": []}], "RandomSeed": -1, "RunTimeoutSeconds": 16.7, "MemoryLimitKB": 64}'

# Using KalmanFilter

In [11]:
class ModelCalibUKF (ModelCalibration, UnscentedKalmanFilter):
    def __init__ (self, model):
        ModelCalibration.__init__ (self, model)
        ModelCalibration.Print(self)
        
        self.measurements = []
        for e in self.model.evaluations:
            if e.GetValue() is None: continue
            assert e.GetError()>0
            self.measurements.append((e.GetValue(),e.GetError()))
        print(f'L={len(self.cpars)} M={len(self.measurements)}')

In [12]:
ckf = ModelCalibUKF(model)


2 calibration parameters:
BrownianMotion eq=1 narg=-1 {"value": 0.11, "step": 0.1, "min": -100, "max": 100}
BrownianMotion eq=3 narg=1 {"value": 2.22, "step": 0.2}
L=2 M=0


In [13]:
model.evaluations

[EvaluationPoint(state=0,time=1),
 EvaluationPoint(state=0,time=2),
 EvaluationPoint(state=0,time=5),
 EvaluationPoint(state=0,time=10)]